In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

c:\Users\giuli\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
with open('matrix_list.txt', 'r') as file:
    content = file.read()

In [3]:
content_list = content.split("\n")
grouped_list = [content_list[i:i+3] for i in range(0, len(content_list), 3)]

In [4]:
samples_prefixes = [sample[0].split("_")[1] for sample in grouped_list[::2]]

In [5]:
samples_prefixes = [f'{sample}_CSF_' for sample in samples_prefixes] + [f'{sample}_PBMC_' for sample in samples_prefixes]

In [6]:
patients = [sample[0].split("_")[0] for sample in grouped_list[::1]]

In [7]:
adatas = {}
for (barcode, feature, matrix), prefix in zip(grouped_list, samples_prefixes):
    adatas[prefix] = sc.read_10x_mtx(
        '../GSE194078/MTX',
        var_names='gene_symbols',
        gex_only=False,
        cache=True,
        prefix=prefix)

In [8]:
csf_adata = adatas[samples_prefixes[0]]

for i, prefix in enumerate(samples_prefixes[1:11]):
    adata = adatas[prefix]
    adata.obs.index = adata.obs.index.str[:-1] + str(i+2)
    adata.var_names_make_unique()
    csf_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)
    
csf_adata

C:\Users\giuli\AppData\Local\Temp\ipykernel_39652\3220132605.py:7: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  csf_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_39652\3220132605.py:7: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  csf_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_39652\3220132605.py:7: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedoc

AnnData object with n_obs × n_vars = 48662 × 36601
    obs: 'batch'
    var: 'gene_ids', 'feature_types'

In [9]:
csf_adata.var['gene_ids']

MIR1302-2HG    ENSG00000243485
FAM138A        ENSG00000237613
OR4F5          ENSG00000186092
AL627309.1     ENSG00000238009
AL627309.3     ENSG00000239945
                    ...       
AC141272.1     ENSG00000277836
AC023491.2     ENSG00000278633
AC007325.1     ENSG00000276017
AC007325.4     ENSG00000278817
AC007325.2     ENSG00000277196
Name: gene_ids, Length: 36601, dtype: object

In [10]:
csf_adata.var['feature_types']

MIR1302-2HG    Gene Expression
FAM138A        Gene Expression
OR4F5          Gene Expression
AL627309.1     Gene Expression
AL627309.3     Gene Expression
                    ...       
AC141272.1     Gene Expression
AC023491.2     Gene Expression
AC007325.1     Gene Expression
AC007325.4     Gene Expression
AC007325.2     Gene Expression
Name: feature_types, Length: 36601, dtype: object

In [11]:
csf_adata.X

<48662x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 100791460 stored elements in Compressed Sparse Row format>

In [13]:
pbmc_adata = adatas[samples_prefixes[11]]

for prefix in samples_prefixes[11:]:
    adata = adatas[prefix]
    adata.obs.index = adata.obs.index.str[:-1] + str(i+2)
    adata.var_names_make_unique()
    pbmc_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)

pbmc_adata

C:\Users\giuli\AppData\Local\Temp\ipykernel_39652\15666685.py:7: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  pbmc_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_39652\15666685.py:7: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  pbmc_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)
c:\Users\giuli\anaconda3\lib\site-packages\anndata\_core\anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
c:\Users\giuli\anaconda3\lib\site-packages\anndata\_

AnnData object with n_obs × n_vars = 61903 × 36601
    obs: 'batch'
    var: 'gene_ids', 'feature_types'

In [14]:
pbmc_adata.X

<61903x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 120415562 stored elements in Compressed Sparse Row format>

In [15]:
sum(csf_adata.var.index == pbmc_adata.var.index) # over 36601

36601

In [16]:
pbmc_adata.var_names_make_unique()
csf_adata.var_names_make_unique()

In [17]:
pbmc_adata.obs.index = pbmc_adata.obs.index + '-PBMC'
csf_adata.obs.index = csf_adata.obs.index + '-CSF'

In [25]:
adata = pbmc_adata.concatenate(csf_adata, join='outer', index_unique=None)
adata.obs_names_make_unique()

C:\Users\giuli\AppData\Local\Temp\ipykernel_39652\1302245356.py:1: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata = pbmc_adata.concatenate(csf_adata, join='outer', index_unique=None)
c:\Users\giuli\anaconda3\lib\site-packages\anndata\_core\anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
c:\Users\giuli\anaconda3\lib\site-packages\anndata\_core\anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
c:\Users\giuli\anaconda3\lib\site-packages\anndata\_core\anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")

In [26]:
adata.obs

,batch
AAACCCACAAGACCGA-1-PBMC,0
AAACCCACACGGCACT-1-PBMC,0
AAACCCATCAGAATAG-1-PBMC,0
AAACGAAAGGTAAAGG-1-PBMC,0
AAACGAACACAGCCAC-1-PBMC,0
...,...
TTTGGTTGTGTTCAGT-11-CSF,1
TTTGGTTTCAAGAAAC-11-CSF,1
TTTGTTGGTATACCCA-11-CSF,1
TTTGTTGGTTATGACC-11-CSF,1


In [27]:
adata.X

<110565x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 221207022 stored elements in Compressed Sparse Row format>

In [28]:
adata.obs.index.value_counts() # make sure that the index is unique

AAACCCACAAGACCGA-1-PBMC    1
CATTCCGGTATGCGTT-4-CSF     1
CATTCCGGTAATGTGA-4-CSF     1
CATTCCGGTAATGCTC-4-CSF     1
CATTCCGCATGACTGT-4-CSF     1
                          ..
GGGCGTTAGATTGTGA-6-PBMC    1
GGGCCATTCTGGCCGA-6-PBMC    1
GGGCCATTCGAATGCT-6-PBMC    1
GGGCCATTCCATCAGA-6-PBMC    1
TTTGTTGTCTCCTGTG-11-CSF    1
Name: count, Length: 110565, dtype: int64

In [29]:
adata.obs[adata.obs.index.value_counts()>1].index.value_counts()

C:\Users\giuli\AppData\Local\Temp\ipykernel_39652\1487988141.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  adata.obs[adata.obs.index.value_counts()>1].index.value_counts()


Series([], Name: count, dtype: int64)

In [30]:
adata.write('GSE194078.h5ad')